```
Daddy told me about cool MD5 hash collision today.
I wanna do something like that too!

ssh col@pwnable.kr -p2222 (pw:guest)
```

In [2]:
import paramiko

class SSHClient:
    def __init__(self, host, user, password, port=22):
        self.host = host
        self.user = user
        self.password = password
        self.port = port
        self.client = paramiko.SSHClient()
        self.client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        self.client.connect(self.host, username=self.user, password=self.password, port=self.port)
        
    def Run(self, cmd):
        (stdin, stdout, stderr) = self.client.exec_command(cmd)
        data = stdout.read().decode()
        return data
    
    def Close(self):
        self.client.close()

In [4]:
host = "pwnable.kr"
user = "col"
port = 2222
password = "guest"

pwnable = SSHClient(host, user, password, port)

In [5]:
cmd = "id"
result = pwnable.Run(cmd)
print(result)

uid=1005(col) gid=1005(col) groups=1005(col)



In [6]:
cmd = "ls -al"
result = pwnable.Run(cmd)
print(result)

total 36
drwxr-x---   5 root    col     4096 Oct 23  2016 .
drwxr-xr-x 114 root    root    4096 May 19 15:59 ..
d---------   2 root    root    4096 Jun 12  2014 .bash_history
-r-sr-x---   1 col_pwn col     7341 Jun 11  2014 col
-rw-r--r--   1 root    root     555 Jun 12  2014 col.c
-r--r-----   1 col_pwn col_pwn   52 Jun 11  2014 flag
dr-xr-xr-x   2 root    root    4096 Aug 20  2014 .irssi
drwxr-xr-x   2 root    root    4096 Oct 23  2016 .pwntools-cache



In [8]:
cmd = "cat col.c"
result = pwnable.Run(cmd)
print(result)

#include <stdio.h>
#include <string.h>
unsigned long hashcode = 0x21DD09EC;
unsigned long check_password(const char* p){
	int* ip = (int*)p;
	int i;
	int res=0;
	for(i=0; i<5; i++){
		res += ip[i];
	}
	return res;
}

int main(int argc, char* argv[]){
	if(argc<2){
		printf("usage : %s [passcode]\n", argv[0]);
		return 0;
	}
	if(strlen(argv[1]) != 20){
		printf("passcode length should be 20 bytes\n");
		return 0;
	}

	if(hashcode == check_password( argv[1] )){
		system("/bin/cat flag");
		return 0;
	}
	else
		printf("wrong passcode.\n");
	return 0;
}



In [70]:
hashcode = 0x121DD09EC
x = b'\x01\x01\x23\x34'
y = (hashcode - 4*(int.from_bytes(x, "little"))).to_bytes(4, "little")
passcode = x*4 + y
print(len(passcode), passcode)
payload = str()
for byte in passcode:
    payload += "\\x{:02x}".format(byte)
print(payload)

20 b'\x01\x01#4\x01\x01#4\x01\x01#4\x01\x01#4\xe8\x05QQ'
\x01\x01\x23\x34\x01\x01\x23\x34\x01\x01\x23\x34\x01\x01\x23\x34\xe8\x05\x51\x51


In [71]:
cmd = "./{} $(echo -e '{}')".format(user, payload)
print(cmd)
result = pwnable.Run(cmd)
print(result)

./col $(echo -e '\x01\x01\x23\x34\x01\x01\x23\x34\x01\x01\x23\x34\x01\x01\x23\x34\xe8\x05\x51\x51')
daddy! I just managed to create a hash collision :)

